In [1]:
import os
import pyspark

myConf=pyspark.SparkConf()
spark=pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
marks=[
    "김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"
]

# 1-1 성적데이터로 DataFrame을 생성.

In [3]:
markRdd=spark.sparkContext.parallelize(marks).map(lambda x:x.split(','))

In [21]:
markDf=spark.createDataFrame(markRdd)

In [53]:
markDf = markDf.withColumnRenamed('_1','name')
markDf = markDf.withColumnRenamed('_2','subject')
markDf = markDf.withColumnRenamed('_3','mark')
markDf.show()

+------+--------+-----+-----------+
|  name| subject|mark2|    zscore2|
+------+--------+-----+-----------+
|김하나| English|100.0| 0.90201485|
|김하나|    Math| 80.0|-0.66217273|
|임하나| English| 70.0| -1.4442666|
|임하나|    Math|100.0| 0.90201485|
|김갑돌| English| 82.3|-0.48229116|
|김갑돌|    Math| 98.5| 0.78470075|
+------+--------+-----+-----------+



# 1-2 zscore 컬럼을 생성.

In [25]:
markDf = markDf.withColumn('mark2',markDf.mark.cast("double"))

In [32]:
import numpy as np
from pyspark.sql import functions as F

_mark= markDf.select("mark2").collect()
_markMean= np.mean(_mark)
_markStd= np.std(_mark, ddof =1)

print(hm)
print(hstd)

88.46666666666665
12.786190467323198


In [34]:
markDf=markDf.select("name","subject","mark2",((markDf.mark2-_markMean)/_markStd).alias("zscore"))

In [40]:
markDf = markDf.withColumn('zscore2',markDf.zscore.cast("float"))

In [51]:
markDf = markDf.drop('zscore')

In [52]:
markDf.show()

+------+--------+-----+-----------+
|  name| subject|mark2|    zscore2|
+------+--------+-----+-----------+
|김하나| English|100.0| 0.90201485|
|김하나|    Math| 80.0|-0.66217273|
|임하나| English| 70.0| -1.4442666|
|임하나|    Math|100.0| 0.90201485|
|김갑돌| English| 82.3|-0.48229116|
|김갑돌|    Math| 98.5| 0.78470075|
+------+--------+-----+-----------+



# 1-3 cdf 컬럼을 생성.